In [1]:
import sys
import os
import time
import pyvisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binascii import unhexlify
import serial

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Drivers")

from Driver_TECSource_5240 import *
from Driver_PM_PM5020 import *
from Driver_OSA_MS9740B import *
from Driver_LaserSource_71440108 import *

In [2]:
rm = pyvisa.ResourceManager()

tec = TECSource_5240()
tec.open(rm,'ASRL7::INSTR')

pm = PM_PM5020()
pm.open(rm)

osa = OSA_MS9740B()
osa.open(rm)

ld = LaserSource_71440108()
ld.open(rm)

OSA Attenuation Status:  ON



## Initialization

In [3]:
ld.set_output_off(1)
ld.set_output_off(2)
ld.set_output_off(3)
ld.set_output_off(4)

In [27]:
T_Start = 25
T_Stop = 50
T_Step = 12.5
t_wait = 100
n = int(((T_Stop-T_Start)/T_Step)+1)
pm.set_units(1,"W")

wavelength_start = 1565
wavelength_stop = 1585
wavelength_points = 2001
OSA_Resolution = 0.03
OSA_VBW = 1000
timeout = 10000

bias_start = 0
bias_stop = 500
bias_step = 2
bias_current = np.linspace(bias_start,bias_stop,int((bias_stop-bias_start)/bias_step) + 1)

Light_engine = "22-49-005"
Channel = 3

osa.set_timeout(20000)

## Function Definition

In [28]:
def sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW):
    osa.set_wavelength(wavelength_start, wavelength_stop, wavelength_points)
    osa.set_resolution_VBW(OSA_Resolution,OSA_VBW)
    osa.sweep_single()
    Power_list = osa.get_sweep_result()
    Power_list_uW = [None] * len(Power_list)
    for i in range(0,len(Power_list)):
        Power_list_uW[i] = 10**(float(Power_list[i])/10)*1000 
    Wavelength_list = np.linspace(wavelength_start,wavelength_stop,wavelength_points)
    Wavelength_peak_nm, Power_peak_dBm = osa.get_peak()   
    if Power_peak_dBm >= -30:
        SMSR_Linewidth_nm, SMSR_dB = osa.get_SMSR()
        Linewidth_3db_nm = osa.get_linewidth(3)
        Linewidth_20db_nm = osa.get_linewidth(20)
    else:
        SMSR_Linewidth_nm, SMSR_dB = 0, 0
        Linewidth_3db_nm = 0
        Linewidth_20db_nm = 0
    return  Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm

## Measurement Routine

In [29]:
meas_df = pd.DataFrame()

In [30]:
for k in range(0,n,1):
    ld.set_laser_current(Channel,0)
    T_Set = T_Start+k*T_Step
    tec.set_temperature(T_Set)
    time.sleep(t_wait)
    for j in range(0,int(((bias_stop-bias_start)/bias_step)+1),bias_step):     
        ld.set_laser_current(Channel,bias_current[j])
        T_read = tec.get_temperature()
        Voltage_V = ld.get_laser_voltage(Channel) 
        Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm = sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW)
        meas_df_temp = pd.DataFrame({'Light_engine':Light_engine, 'Channel':Channel, 'Bias_current_mA':bias_current[j], 'Voltage_V':Voltage_V, 'Temeprature_C':T_read, 'Wavelength_nm':Wavelength_list, 'Power_dBm':Power_list, 
                                    'Power_uW':Power_list_uW, 'Wavelength_peak_nm':Wavelength_peak_nm, 'Power_peak_nm':Power_peak_dBm, 'SMSR_dB':SMSR_dB, 'SMSR_linewidth_nm':SMSR_Linewidth_nm,
                                    '3dB_linewidth_nm':Linewidth_3db_nm, '20dB_linewidth_nm':Linewidth_20db_nm})
        data = [meas_df_temp, meas_df]
        meas_df = pd.concat(data,ignore_index=True,sort=False)

In [31]:
meas_df.to_csv(r"22-49-005_Channel_3.csv")

In [32]:
meas_df

,Light_engine,Channel,Bias_current_mA,Voltage_V,Temeprature_C,Wavelength_nm,Power_dBm,Power_uW,Wavelength_peak_nm,Power_peak_nm,SMSR_dB,SMSR_linewidth_nm,3dB_linewidth_nm,20dB_linewidth_nm
0,22-49-005,3,500.0,2.694,50.01,1565.00,-50.36,9.204496e-03,1580.49,17.07,60.35,0.45,0.082,0.082
1,22-49-005,3,500.0,2.694,50.01,1565.01,-52.66,5.420009e-03,1580.49,17.07,60.35,0.45,0.082,0.082
2,22-49-005,3,500.0,2.694,50.01,1565.02,-52.14,6.109420e-03,1580.49,17.07,60.35,0.45,0.082,0.082
3,22-49-005,3,500.0,2.694,50.01,1565.03,-50.67,8.570378e-03,1580.49,17.07,60.35,0.45,0.082,0.082
4,22-49-005,3,500.0,2.694,50.01,1565.04,-52.68,5.395106e-03,1580.49,17.07,60.35,0.45,0.082,0.082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756373,22-49-005,3,0.0,0.002,25.00,1584.96,-63.16,4.830588e-04,1568.81,-54.78,0.00,0.00,0.000,0.000
756374,22-49-005,3,0.0,0.002,25.00,1584.97,-60.35,9.225714e-04,1568.81,-54.78,0.00,0.00,0.000,0.000
756375,22-49-005,3,0.0,0.002,25.00,1584.98,-120.0,1.000000e-09,1568.81,-54.78,0.00,0.00,0.000,0.000
756376,22-49-005,3,0.0,0.002,25.00,1584.99,-120.0,1.000000e-09,1568.81,-54.78,0.00,0.00,0.000,0.000


In [33]:
tec.set_temperature(25)
ld.set_laser_current(3,0)

In [34]:
tec.set_output_off()

In [35]:
pm.close()
tec.close()
osa.close()
ld.close()